In [1]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration, MusicgenProcessor, MusicgenForCausalLM
from peft.config import PeftConfig
from peft import get_peft_config, get_peft_model, PeftModel

/home/rhw/.pyenv/versions/3.10.12/envs/musicgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = PeftConfig.from_pretrained("tuned-musicgen-small-lora")

In [3]:

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

full_model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
decoder = PeftModel.from_pretrained(full_model.decoder, "tuned-musicgen-small-lora")
full_model.decoder = decoder


In [4]:
full_model = full_model.to("cuda")
full_model.eval()

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [17]:

inputs = processor(
    text=[
        "slow sad phonk with mixed instruments",
        "2023s energetic epic phonk",
        '80s phonk track with bassy drums and synth',],
    padding=True,
    return_tensors="pt",
)
inputs = {k:v.to("cuda") for k, v in inputs.items()}

audio_values = full_model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=756)

In [18]:
# audio_values = processor.batch_decode(audio_values, padding_mask=inputs["padding_mask"])

In [19]:
from audiocraft.utils.notebook import display_audio

In [22]:
display_audio(audio_values[0], sample_rate=44100)